# Problem

This dataset is a record of 7 common different fish species in fish market sales. We want a model to estimate a fish's weight. 

https://www.kaggle.com/aungpyaeap/fish-market

# Summary of Results

Linear regression performed with an r2 score of `0.965` on the test set while fairly well satisfying the assumptions of an OLS estimator.

***

In [ ]:
!pip install kaggle

In [ ]:
!pip install sklearn2pmml

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import kaggle

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

sns.set_theme(style="darkgrid")
%matplotlib inline

# Exploratory Data Analysis

In [ ]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files('aungpyaeap/fish-market', path='data/', unzip=True)

## Data Dictionary

* Species: Species name of fish
* Weight: Weight of fish in gram
* Length1: Vertical length in cm
* Length2: Diagonal length in cm
* Length3: Cross length in cm
* Height: Height in cm
* Width: Diagonal width in cm

In [ ]:
data = pd.read_csv('./data/Fish.csv')

## Overview

First, we gather basic impressions and answer basic questions abou the data like,

* What do some sample values look like?
* How many rows are there and what are their types?

In [ ]:
data.sample(frac=.1).head(10)

In [ ]:
data.info();

In [ ]:
data.describe().T

## Data Quality

* Is there missing data?
* Are columns the right types?
* Are there outliers in any of the columns? Consider uni-variate and multi-variate analysis. 

#### *Is there missing data?*

In [ ]:
print(str('Are there any missing values in the dataset?'), data.isnull().values.any())

#### *Are columns the right types?*

In [ ]:
pd.DataFrame(data['Species'].value_counts()).T

We can see from the data type displayed in the overview section and the data above that the `Species` columns is categorical.

In [ ]:
 data['Species'] = data['Species'].astype('category')

#### *Are there outliers in any of the columns?*

**Univariate Analysis** 

In [ ]:
boxplot_fields = ['Weight','Length1','Length2','Length3','Height','Width']
fig, ax = plt.subplots(2, 3, figsize=(30, 18))
for var, subplot in zip(boxplot_fields, ax.flatten()):
    sns.boxplot(x=var, data=data, ax=subplot)

**Multi-Variate Analysis**

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(30, 18))
for var, subplot in zip(boxplot_fields, ax.flatten()):
    sns.boxplot(x='Species', y=var, data=data, ax=subplot)

In [ ]:
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

lowerend = 0 # This would otherwise be Q1[c] - (1.5 * IQR[c]) but negative values don't make sense for the variables.
upperend = Q3 + (1.5 * IQR)

for c in boxplot_fields:
    print(c + ':')
    upperend = Q3[c] + (1.5 * IQR[c])
    feature = data[c]
    outliers = feature[(feature <= lowerend) | (feature > upperend)]
    print(outliers)

In [ ]:
fishes = data[~((data[boxplot_fields] <= 0) | (data[boxplot_fields] > (Q3 + 1.5 * IQR))).any(axis=1)]

*Observations*
* The outliers that exist per feature in the univariate analysis are not all the same ones that exist in the multivariate analysis by species. So if outliers are identified by a categorical feature that will be in one's model is it often better to remove the overall outliers or those by class of a given categorical feature?

## Graphical Exploration

#### Distributional Observations

An early step in any effort to analyze or model data is understanding how the variables are distributed.

* What range do the observations cover? 
* What is their central tendency? 
* Are they heavily skewed in one direction? 
* Is there evidence for bimodality? 
* Are there significant outliers? 
* Do the answers to these questions vary across subsets defined by other variables?
* Is the response variable imbalanced?

In [ ]:
plt.figure(figsize=(12,6))
ax = sns.countplot(x="Species", data=fishes);
ax.set_ylabel("Count")
plt.show()

*Observations*
* If the response variable were `Species` then this plot would tell use that we have an imbalanced dataset.

In [ ]:
columns_for_hist = ['Weight','Length1','Length2','Length3','Height','Width']
fig, ax = plt.subplots(2, 3, squeeze=True, figsize=(30, 18))
for col, subplot in zip(columns_for_hist, ax.flatten()):
    sns.histplot(fishes, x=col, ax=subplot);

*Observations*
* Length[1,2,3] have nearly identical distributions, which identifies them as probably multicolinear. Thus not all of them should be included in a linear model.
* Weight resembles a log normal distribution so transforming that would be worth trying in a our model

#### Relationships

First I'll explore correlations. In statistical terms, correlation is a method of assessing a possible two-way linear association between two continuous variables. 

**Pearson's Correlation**

The Pearson product-moment correlation attempts to draw a line of best fit through the data of two variables. The Pearson correlation coefficient, *r*, indicates how far away all these data points are to this line of best fit (i.e., how well the data points fit this new model/line of best fit). The key assumptions of using this statistic are,

* Both variables being studied are normally distributed
* This coefficient is affected by extreme values, which may exaggerate or dampen the strength of relationship, and is therefore inappropriate when either or both variables are not normally distributed.

The condition of normal distribution isn't well satisfied from what we can see in the histograms above, but I examine the values for the purposes of understanding our problem.

In [ ]:
plt.figure(figsize=(12,8))
plt.title("Pearson's Correlation")
sns.heatmap(fishes.corr(), annot=True);
plt.show();

*Observations*
* The response variable, Weight, is highly correlated with all of the other numerical features, which is an indication of high multicolinearity. Although, not definitely.

**Spearmen's Correlation**



The key assumptions of using this statistic are,

* It is appropriate when one or both variables are skewed or ordinal1 and is robust when extreme values are present.

In [ ]:
plt.figure(figsize=(12,8))
plt.title("Spearman's Correlation")
sns.heatmap(fishes.corr(method='spearman'), annot=True);
plt.show();

*Observations*
* In contrast to Pearson's correlation we see that highest correlation of `Weight` between `Width` and any of `Length[1,2,3]`, but all features show high correlation coefficents. 
* Features `Length[1,2,3]` are highly colinear 

Lastly, I examine the relationships amongst the variables using a scatter plot.

In [ ]:
g = sns.pairplot(fishes, kind='scatter', hue='Species');

*Observations*

* There continues to be strong linear relationships between our response variable, `Weight`, and all the features generally.
* There are strong linear relationships between `Weight` and each of the features by `Species`.

# The Model

We saw from Spearmen's correlation and the pair plot above that there are strong linear relationships betweeen `Weight` and serveral of the other variables. 

**Using Statsmodel**

In [ ]:
from statsmodels.formula.api import ols

train, test = train_test_split(fishes, test_size=0.3, random_state=1)

mod = ols(formula='np.log(Weight) ~ np.log(Width) ', data=train)
res = mod.fit()

res.summary()

The following models were also tried, but I found they didn't score as well or violated the assumptions of linear models to a degree that didn't justify their use.
* Weight ~ Width
* Weight ~ Height * Width * Length1
* Weight ~ (Height * Width * Length1) + C(Species)
* np.log(Weight) ~ np.log(Height) * np.log(Length1) * np.log(Width) + C(Species)
* np.log(Weight) ~ np.log(Height) + np.log(Length1) + np.log(Width) + C(Species)

*Test Set Performance*

In [ ]:
X_test = test.drop(columns=['Weight'])
y_test_log = np.log(test['Weight'])
y_pred = res.predict(X_test)

resids = y_test_log - y_pred

In [ ]:
print('Coefficient of determination: %.3f' % r2_score(y_test_log, y_pred))

In [ ]:
print('Mean squared error: %.2f' % mean_squared_error(y_test_log, y_pred))

**Validate Assumptions of Linear Regression**

*Validate Linearity*

To detect nonlinearity one can inspect plots of actual vs. predicted values or residuals vs. predicted values. The desired outcome is that points are symmetrically distributed around a diagonal line in the former plot or around a horizontal line in the latter one. In both cases with a roughly constant variance.

In [ ]:
ax = sns.regplot(x=y_pred, y=y_test_log, lowess=True, line_kws={'color': 'red'})
ax.set_title('Actual vs. Predicted Values', fontsize=16)
ax.set(xlabel='Predicted', ylabel='Actual');

*Residuals Normally Distributed*

The qualtile-quantile (Q-Q) plot provides a handy visual means to inspect the similarity of distributions of a data set. The idea is to plot the quantiles of the sample on the vertical axis and the quantiles of the thoretical distribution on the horizontal axis. If the points of the plot fall on an approximately straight line, you can conclude that the sample distribution is close to the thoretical.

In [ ]:
from scipy import stats

plt.subplots(figsize=(15, 12))
ax1 = plt.subplot(221) 
stats.probplot(resids, plot=ax1)
ax1.set_title('Probability Plot', fontsize=16)
ax1 = plt.subplot(222) 
sns.distplot(resids, ax=ax1)
ax1.set_title('Distribution of Residuals', fontsize=16)
plt.show()

Next we apply a formal method. The Anderson-Darling test for normal distribution unknown mean and variance. Passing the normality test only allows you to state no significant departure from normality was found.

In [ ]:
from statsmodels.stats.diagnostic import normal_ad

p_value = normal_ad(resids)[1]

print('p-value from the test - below 0.05 generally means non-normal:', p_value)

*No Multicollinearity Among Predictors*

Since we only have one predictor this requirement is not a concern for this model.

*No Autocorrelation of the Error Terms*

In [ ]:
from statsmodels.stats.stattools import durbin_watson

print('\nPerforming Durbin-Watson Test')
print('Values of 1.5 < d < 2.5 generally show that there is no autocorrelation in the data')
print('0 to 2< is positive autocorrelation')
print('>2 to 4 is negative autocorrelation')
print('-------------------------------------')

durbinWatson = durbin_watson(resids)
print('Durbin-Watson:', durbinWatson)
if durbinWatson < 1.5:
    print('Signs of positive autocorrelation', '\n')
    print('Assumption not satisfied')
elif durbinWatson > 2.5:
    print('Signs of negative autocorrelation', '\n')
    print('Assumption not satisfied')
else:
    print('Little to no autocorrelation', '\n')
    print('Assumption satisfied')

*Homoscedasticity*

This assumes homoscedasticity, which is the same variance within our error terms. Heteroscedasticity, the violation of homoscedasticity, occurs when we don’t have an even variance across the error terms.

In [ ]:
sns.residplot(x=np.log(X_test['Width']), y=y_test_log, lowess=True, color="b");

The plot clearly shows there's not a uniform variance, but it is better than many of the other models that were list above.

**Using Scikit-Learn (using LinearRegression)**

Below we try out the Scikit-Learn implementation in order to compare.

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor

from sklearn2pmml import sklearn2pmml
from sklearn_pandas import DataFrameMapper
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml.decoration import ContinuousDomain
from sklearn2pmml.preprocessing import ExpressionTransformer

X = fishes.filter(items=['Width'])
y = np.log(fishes['Weight'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


mapper = DataFrameMapper([
    (["Width"], [ContinuousDomain(), ExpressionTransformer("numpy.log(X[0])", dtype = np.float64)])
    #(["Width"], [ContinuousDomain(), FunctionTransformer(np.log)])
    #("Width", FunctionTransformer(np.log))    
])
mapper.fit_transform(X)

model_pipeline = PMMLPipeline([
    ("mapper", mapper),
    #("model", TransformedTargetRegressor(regressor=LinearRegression(), func=np.log, inverse_func=np.exp))
    ("model", LinearRegression())
])


clf = model_pipeline.fit(X_train, y_train);
sklearn2pmml(clf, 'fish-weight-model.pmml', with_repr=True, debug=True)

*Test Set Performance*

In [ ]:
# Make predictions using the testing set
y_pred = clf.predict(X_test)

In [ ]:
print('Coefficient of determination: %.3f' % r2_score(y_test, y_pred))

In [ ]:
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))

# Conclusion

Linear regression performed with an r2 score of `0.965` on the test set while fairly well satisfying the assumptions of an OLS estimator.